In [1]:
import pylab
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from scipy.spatial import distance
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from mpl_toolkits.axes_grid1 import make_axes_locatable
import string

In [2]:
### Model structure

## Tactile area
xt_dim = 10
yt_dim = 5

# Tactile neurons
Mt = 20
Nt = 10

# Tactile RF centres
xt = np.arange(1,Mt+1)*0.5 
yt = np.arange(1,Nt+1)*0.5

## Auditory area
xa_dim = 200
ya_dim = 30

# Auditory neurons
Ma = 20
Na = 3

# Auditory RF centres
xa = (np.arange(1,Ma+1)*10)-5
ya = (np.arange(1,Na+1)*10)-15

In [3]:
### Model fixed parameters

## Unisensory receptive fields
phit_0 = 1
sigmat_phi = 0.5
phia_0 = 1
sigmaa_phi = 10

## Neuronal activity

# Tactile neurons
ft_min = -0.12
ft_max = 1
qt_c = 19.43
rt = 0.34

# Auditory neurons
fa_min = -0.12
fa_max = 1
qa_c = 19.43
ra = 0.34

# Multisensory neuron
fm_min = 0
fm_max = 1
qm_c = 12
rm = 0.6

tau = 20

## External stimuli

# Tactile stimuli
it_0 = 2.5
sigmat_i = 0.3

xt_0 = 5 
yt_0 = 2.5
tt_0 = 0

# Auditory stimuli
ia_0 = 3.6
sigmaa_i = 0.3

ya_0 = 5
xa_0 = 100
ta_0 = 0

In [4]:
## External stimulus
def stimt(x,y,t):
    if t<tt_0: 
        I = 0
    else: 
        I = (it_0)*np.exp(- (np.square(xt_0-x)+np.square(yt_0-y))/(2*np.square(sigmat_i)))
    return I 

def stima(x,y,t,xa_0):
    if t<ta_0: 
        I = 0
    else: 
        I = (ia_0)*np.exp(- (np.square(xa_0-x)+np.square(ya_0-y))/(2*np.square(sigmaa_i)))
    return I 

In [5]:
### Receptive fields

## Functions

# Tactile area
def phit(x,y):
    phi = np.zeros((Mt,Nt))
    for i in range(Mt):
        for j in range(Nt):
            phi[i][j] = phit_0*np.exp(-((np.square(xt[i]-x)+np.square(yt[j]-y))/(2*np.square(sigmat_phi))))
    return phi

# Auditory area
def phia(x,y):
    phi = np.zeros((Ma,Na))
    for i in range(Ma):
        for j in range(Na):
            phi[i][j] = phia_0*np.exp(-((np.square(xa[i]-x)+np.square(ya[j]-y))/(2*np.square(sigmaa_phi))))
    return phi

## Calculation
dif = 0.2

# Tactile area
xt_i = np.arange(0,xt_dim+dif,dif)
yt_n = np.arange(0,yt_dim+dif,dif)

phi_t = np.zeros((Mt,Nt,len(xt_i),len(yt_n)))        
for k in range(len(xt_i)):
    for l in range(len(yt_n)):
        phi_t[:,:,k,l] = phit(xt_i[k],yt_n[l])

# Auditory area        
xa_i = np.arange(0,xa_dim+dif,dif)
ya_n = np.arange(0,ya_dim+dif,dif)

phi_a = np.zeros((Ma,Na,len(xa_i),len(ya_n)))        
for k in range(len(xa_i)):
    for l in range(len(ya_n)):
        phi_a[:,:,k,l] = phia(xa_i[k],ya_n[l])

In [ ]:
### Synapses

## Lateral synapses in unisensory areas

def Lw(Lt_ex,Lt_in,sigmat_ex,sigmat_in,La_ex,La_in,sigmaa_ex,sigmaa_in):
    # Tactile Connections
    Lt = np.zeros((Mt*Nt,Mt*Nt))

    for i in range(Mt*Nt):
        for j in range(Mt*Nt):
            if i==j: 
                Lt[i,j] = 0
            else:
                Dtx = xt[np.floor_divide(i,Nt)] - xt[np.floor_divide(j,Nt)]
                Dty = yt[np.remainder(i,Nt)] - yt[np.remainder(j,Nt)]
                Lt[i,j] = Lt_ex*np.exp(- (np.square(Dtx)+np.square(Dty))/(2*np.square(sigmat_ex)))-Lt_in*np.exp(- (np.square(Dtx)+np.square(Dty))/(2*np.square(sigmat_in)))

    # Auditory Connections
    La = np.zeros((Ma*Na,Ma*Na))

    for i in range(Ma*Na):
        for j in range(Ma*Na):
            if i==j: 
                La[i,j] = 0
            else:
                Dax = xa[np.floor_divide(i,Na)] - xa[np.floor_divide(j,Na)]
                Day = ya[np.remainder(i,Na)] - ya[np.remainder(j,Na)] #before was remainder-1
                La[i,j] = La_ex*np.exp(- (np.square(Dax)+np.square(Day))/(2*np.square(sigmaa_ex)))-La_in*np.exp(- (np.square(Dax)+np.square(Day))/(2*np.square(sigmaa_in)))
    return Lt,La

## Feedforward and feedback synapses

def Fw (Wt_0,Wa_0,Bt_0,Ba_0):
    k1 = 1.67047305e+01
    k2 = 9.82951020e+02
    alpha = 9.82802685e-01
    
    # Tactile connections
    Bt = np.ones((Mt,Nt))*Bt_0
    Wt = np.ones((Mt,Nt))*Wt_0

    # Auditory connections
    Ba = np.zeros((Ma,Na))
    Wa = np.zeros((Ma,Na))

    lim = 4.86791243e+01

    for i in range(Ma):
        for j in range(Na):
            if (xa[i]<lim):
                D = 0
            else: 
                D = distance.euclidean((xa[i],ya[j]),(lim,ya[j]))              
            Ba[i,j] = alpha*Ba_0*np.exp(- D/k1)+(1-alpha)*Ba_0*np.exp(- D/k2)
            Wa[i,j] = alpha*Wa_0*np.exp(- D/k1)+(1-alpha)*Wa_0*np.exp(- D/k2)
    return Wt,Wa,Bt,Ba

In [7]:
### Inputs

## Unisensory input

# Tactile input
def PHIt(t):
        PHI = np.zeros((Mt,Nt,len(xt_i),len(yt_n)))        
        for k in range(len(xt_i)):
            for l in range(len(yt_n)):
                PHI[:,:,k,l] = np.multiply(phi_t[:,:,k,l],stimt(xt_i[k],yt_n[l],t))
        PHI = np.sum(PHI,axis=3)
        PHI = np.sum(PHI,axis=2)
        return PHI    

# Auditory input
def PHIa(t,xa_0):

        PHI = np.zeros((Ma,Na,len(xa_i),len(ya_n)))        
        
        for k in range(len(xa_i)):
            for l in range(len(ya_n)):
                PHI[:,:,k,l] = np.multiply(phi_a[:,:,k,l],stima(xa_i[k],ya_n[l],t,xa_0))
        PHI = np.sum(PHI,axis=3)
        PHI = np.sum(PHI,axis=2)
        return PHI
    
## Lateral inputs
# Tactile area
def LIt(z,Lt): # z is a matrix of MtxNt dimensions
    LI = np.zeros(Mt*Nt)
    z = np.reshape(z,(1,Mt*Nt))
    for i in range(Mt*Nt):
            LI[i] = np.sum(np.multiply(Lt[i,:],z[0,:])) 
    LI = np.reshape(LI,(Mt,Nt))
    return LI

# Auditory area
def LIa(z,La): # z is a matrix of MaxNa dimensions
    LI = np.zeros(Ma*Na)
    z = np.reshape(z,(1,Ma*Na))
    for i in range(Ma*Na):
            LI[i] = np.sum(np.multiply(La[i,:],z[0,:])) 
    LI = np.reshape(LI,(Ma,Na))
    return LI

## Feedback inputs

# Tactile area
def bt(z,Bt):
    bt = np.multiply(Bt,z)
    return bt

# Auditory area
def ba(z,Ba):
    ba = np.multiply(Ba,z)
    return ba

In [8]:
### Neuronal activity 

## Tactile neurons
def psit(qt): # the parameter b stands for the bias term introduced to generate E/I imbalance
    y = qt
    for i in range(Mt):
        for j in range(Nt):
            y[i,j] = (ft_min+ft_max*np.exp((qt[i,j]-qt_c)*rt))/(1+np.exp((qt[i,j]-qt_c)*rt))
    return y

## Auditory neurons
def psia(qa): # the parameter b stands for the bias term introduced to generate E/I imbalance
    y = qa
    for i in range(Ma):
        for j in range(Na):
            y[i,j] = (fa_min+fa_max*np.exp((qa[i,j]-qa_c)*ra))/(1+np.exp((qa[i,j]-qa_c)*ra))
    return y

## Multisensory neuron
def psim(qm):
    y = (fm_min+fm_max*np.exp((qm-qm_c)*rm))/(1+np.exp((qm-qm_c)*rm))
    return y


In [9]:
### Evolutionary prunning mechanism

def prun(WM,pr):
    newM = np.copy(WM)
    newM[newM < pr] = 0
    return newM

In [21]:
### Audio-tactile experiment 

## Experiment function
def experimentrun(a_distances,time,b,pr,Lex,Lin,Sex,Sin):
    Lt,La = Lw(Lex,Lin,Sex,Sin,Lex,Lin,Sex*20,Sin*20)
    Wt,Wa,Bt,Ba = Fw(6.5,6.5,b,b)
    dt = 0.4
    timesteps = int(time/dt)
    ndist = len(a_distances)
    
    RTs = np.zeros(ndist)
    ZTs = np.zeros((Mt,Nt,ndist))
    ZAs = np.zeros((Ma,Na,ndist))
    
    ti = PHIt(0) 
    PrWa = prun(Wa,pr)
    dtau = dt/tau
    
    qt = np.zeros((Mt,Nt,timesteps+1,ndist))
    ut = np.zeros((Mt,Nt,timesteps+1,ndist))
    zt = np.zeros((Mt,Nt,timesteps+1,ndist))
    pt = np.zeros((Mt,Nt,timesteps+1,ndist))

    qa = np.zeros((Ma,Na,timesteps+1,ndist))
    ua = np.zeros((Ma,Na,timesteps+1,ndist))
    za = np.zeros((Ma,Na,timesteps+1,ndist))
    pa = np.zeros((Ma,Na,timesteps+1,ndist))

    qm = np.zeros((timesteps+1,ndist))
    um = np.zeros((timesteps+1,ndist))
    zm = np.zeros((timesteps+1,ndist))
    pm = np.zeros((timesteps+1,ndist))
    
    rt = np.zeros((timesteps+1,ndist))
    
    for d in range(ndist):
        xa_0 = a_distances[d] # How far the sound is presented.     
        ai = PHIa(0,xa_0) # Generates an auditory input

        for i in range(timesteps):    
            # Tactile activity
            ut[:,:,i+1,d] = ti+LIt(zt[:,:,i,d],Lt)+bt(zm[i,d],Bt)
            qt[:,:,i+1,d] = qt[:,:,i,d] + dtau*(-qt[:,:,i,d]+ut[:,:,i,d])
            pt[:,:,i+1,d] = psit(qt[:,:,i,d])
            zt[:,:,i+1,d] = pt[:,:,i,d]*np.heaviside(pt[:,:,i,d],0)

            # Auditory activity
            ua[:,:,i+1,d] = ai+LIa(za[:,:,i,d],La)+ba(zm[i,d],Ba) 
            qa[:,:,i+1,d] = qa[:,:,i,d] + dtau*(-qa[:,:,i,d]+ua[:,:,i,d])
            pa[:,:,i+1,d] = psia(qa[:,:,i,d])
            za[:,:,i+1,d] = pa[:,:,i,d]*np.heaviside(pa[:,:,i,d],0)

            # Multisensory activity
            um[i+1,d] = np.sum(np.multiply(Wt,zt[:,:,i,d])) + np.sum(np.multiply(PrWa,za[:,:,i,d]))
            qm[i+1,d] = qm[i,d] + dtau*(-qm[i,d]+um[i,d])
            pm[i+1,d] = psim(qm[i,d])
            zm[i+1,d] = pm[i,d]*np.heaviside(pm[i,d],0)
            
            # Response
            rt[i+1,d] = np.any(zt[:,:,i,d]>(0.9*ft_max))
            
        RTs[d] = np.argmax(rt[:,d])*dt
        ZMs = zm        
        ZTs[:,:,d] = zt[:,:,timesteps,d]
        ZAs[:,:,d] = za[:,:,timesteps,d]   
    return RTs,ZMs,ZTs,ZAs

In [ ]:
def RTsig(data,cp,h):
    global ymin
    global ymax
    return (ymin+ymax*np.exp((data-cp)/h)) / (1+np.exp((data-cp)/h))

## Sigmoid function fitting
def sigfit(x,y):
    global ymin
    global ymax
    
    k_0 = (ymax - ymin)/(x[-1]-x[0])
    initial_slope = (ymax - ymin)/(4*k_0)
    middle_x = np.max(x)/2
    init_guess = [middle_x, initial_slope]
    boundaries = ([0,float('-inf')],[float('inf'),float('inf')])
    
    popt, pcov = curve_fit(RTsig,x,y,p0=init_guess,method='trf',ftol=1e-8,xtol=1e-8,maxfev=10000,bounds=boundaries)
    sigpar = np.asarray(popt)
    dc = sigpar[0]
    h = sigpar[1]
    
    return dc,h